In [1]:
from util import *

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']
Add path ../../..
Connected DB - conn
Connected oraDW_ANALYTICS - conn_aly


In [2]:
import os
import numpy as np

In [3]:
import matplotlib.image as mpimg

In [4]:
from IPython.display import HTML

# NAMESPACE

In [5]:
class NAMESPACE:
    # Base values
    TABLE_NAME = 'DW_CARD_TRANSACTION_FCT'
    SEL_SAMPLE = 'SAMPLE1_SUB3'
    
    # Important to look
    QUERY = f"""SELECT X.* FROM DW_ANALYTICS.{TABLE_NAME} X JOIN CINS_TMP_ALY_CUST_{SEL_SAMPLE} SAM ON X.CUSTOMER_CDE = SAM.CUSTOMER_CDE"""
    
    # Derived values
    data_fp = f'./data/{TABLE_NAME}'
    data_sample_nm = os.path.join(data_fp, SEL_SAMPLE)
    IMG_FP = f'./img/{TABLE_NAME}'
    img = f'./img/{TABLE_NAME}/{SEL_SAMPLE}'
    
    def show():
        atts = dir(NAMESPACE)
        for a in atts:
            if not a.startswith('__'):
                print(a)
    
    @classmethod
    def init(cls):
        if not os.path.exists(NAMESPACE.data_fp):
            os.makedirs(NAMESPACE.data_fp)
        if not os.path.exists(NAMESPACE.img):
            os.makedirs(NAMESPACE.img)

In [6]:
NAMESPACE.QUERY

'SELECT X.* FROM DW_ANALYTICS.DW_CARD_TRANSACTION_FCT X JOIN CINS_TMP_ALY_CUST_SAMPLE1_SUB3 SAM ON X.CUSTOMER_CDE = SAM.CUSTOMER_CDE'

In [7]:
NAMESPACE.init()

In [8]:
NAMESPACE.show()

IMG_FP
QUERY
SEL_SAMPLE
TABLE_NAME
data_fp
data_sample_nm
img
init
show


In [9]:
stat_opt = [
            'count',
            'mean',
            'median',
            'min',
            'max',
            'std',
]
ordered_stat_opt = [[i, i + '_pct_chg'] for i in stat_opt]
ordered_stat_opt = [j for i in ordered_stat_opt for j in i]

# Functions

In [10]:
def aly_datetime(df, date_col, cde_col, resample='1Y', outlier_date='1930-01-01', timedelta='365 days',lvl=3):
    display(HTML(f'<h{lvl}>{date_col}</h{lvl}>'))
    if cde_col is None:
        tmp_df = df[date_col].copy()
        tmp_df[date_col] = pd.to_datetime(tmp_df[date_col], errors='coerce')
        tmp_df[date_col] = df[date_col].astype('datetime64[D]')
        tmp = tmp_df[tmp_df[date_col] > outlier_date]
        plot_df = tmp.set_index(date_col).resample(resample).nunique()
    else:
        tmp_df = df[[date_col, cde_col]].copy()
        tmp_df[date_col] = pd.to_datetime(tmp_df[date_col], errors='coerce')
        tmp_df[date_col] = df[date_col].astype('datetime64[D]')
        tmp = tmp_df[tmp_df[date_col] > outlier_date]
        plot_df = tmp.set_index(date_col)[cde_col].resample(resample).nunique()
    
    fig, axs = plt.subplots(1,2, figsize=(15, 5))
    plot_df.plot(kind='bar', rot=70, ax=axs[0])
    up2now = ((datetime(2023,11,1) - tmp[date_col]) / pd.Timedelta(timedelta)).apply(float)
    sns.histplot(up2now, kde=True, ax=axs[1])

In [11]:
def default_func(x):
    return x

In [12]:
def remove_mode_value(x):
    return x[~x.isin(x.mode().values)]

In [13]:
def logx(x):
    return np.log(x)

In [14]:
def plot_multi_hist_aly(df,cols=[],funcs=[default_func, remove_mode_value, logx], lvl=3):
    def df_stat_after_transformed(df_trans):
        def pct_change(series):
            first_e = series.iloc[0]
            pct_chg = (series - first_e) / first_e * 100
            pct_chg.iloc[0] = 100.
            return pct_chg
        stat_infos = []
        for data_trans_series in df_trans:
            # Get stats info
            stat_info = []
            for s in stat_opt:
                if s == 'count':
                    stat_info.append(data_trans_series.count())
                elif s == 'mean':
                    stat_info.append(data_trans_series.mean())
                elif s == 'median':
                    stat_info.append(data_trans_series.median())
                elif s == 'min':
                    stat_info.append(data_trans_series.min())
                elif s == 'max':
                    stat_info.append(data_trans_series.max())
                elif s == 'std':
                    stat_info.append(data_trans_series.std())
            stat_infos.append(stat_info)
        col_stat = stat_opt
        index_stat = [f.__name__ for f in funcs]
        stat_df = pd.DataFrame(data=stat_infos, columns=col_stat, index=index_stat)
        for c in stat_df.columns:
            stat_df[f'{c}_pct_chg'] = pct_change(stat_df[c])
        stat_df = stat_df[ordered_stat_opt]
        result_df = stat_df.T
        return result_df
    
    nrows, ncols = len(cols), len(funcs)
    for i, c in enumerate(cols):
        display(HTML(f'<h{lvl}>{c}</h{lvl}>'))
        if c in df.columns:
            data_trans = []
            for f in funcs:
                pre_data_tran = data_trans[-1] if len(data_trans) > 0 else df[c]
                trans_data = f(pre_data_tran)
                data_trans.append(trans_data)
            # plot section 
            fig, axs = plt.subplots(1,ncols, figsize=(4* ncols, 4 ))
            for j, d in enumerate(data_trans):
                sel_ax = axs[j]
                sel_ax.set_title(f'{cols[i]}\n{funcs[j].__name__}')
                d.replace([np.inf, -np.inf], np.nan).plot(kind='hist',bins=100,ax=sel_ax)
            pd.options.display.float_format = '{:.3f}'.format
            display(df_stat_after_transformed(data_trans))
            plt.tight_layout()
            plt.savefig(os.path.join(NAMESPACE.img, f'{c}.jpg'), dpi=500)
            plt.show()
            print('\n\n\n')

In [15]:
def aly_cat_col(df,cat_cols,k=10, lvl=3):
    for cat_col in cat_cols: 
        display(HTML(f'<h{lvl}>{cat_col}</h{lvl}>'))
        fig,axs=plt.subplots(1,2,figsize=(10,5))
        display(df[cat_col].value_counts())
        df[cat_col].value_counts()[:k].plot(kind='bar', rot=80,ax=axs[0])
        df[cat_col].value_counts()[:k].plot(kind='pie', autopct='%.2f%%',ax=axs[1])
        plt.savefig(os.path.join(NAMESPACE.img, f'{cat_col}.jpg'), dpi=500)
        plt.show()

In [16]:
def download_or_reload(saved_file):
    if not os.path.exists(saved_file):
        query = NAMESPACE.QUERY
        cur.execute(query)
        result = cur.fetchall()
        column_names = [c[0] for c in cur.description]
        df = pd.DataFrame(result, columns=column_names)
        df.to_pickle(saved_file)
    df = pd.read_pickle(saved_file)
    print(f'Len {len(df)}')
    return df

In [17]:
def gen_info_stats(df):
    def info_stats(c):
        dtype = df[c].dtype
        n = len(df)
        null_pct = (n - df[c].count()) / n
        nunique = df[c].nunique()
        nunique_pct = nunique / n
        unique_values = df[c].unique()[:3]
        return c, null_pct, dtype, nunique, nunique_pct, unique_values

    data_stats = []
    for c in df.columns:
        data_stats.append(info_stats(c))
    data_stats = pd.DataFrame(data=data_stats,columns=['Column', 'Null Pct', 'Dtype', 'Nunique', 'Nunique Pct', 'Sample Unique Value'])
    return data_stats

# Sample1-Sub*

In [18]:
NAMESPACE.QUERY

'SELECT X.* FROM DW_ANALYTICS.DW_CARD_TRANSACTION_FCT X JOIN CINS_TMP_ALY_CUST_SAMPLE1_SUB3 SAM ON X.CUSTOMER_CDE = SAM.CUSTOMER_CDE'

In [19]:
df = download_or_reload(NAMESPACE.data_sample_nm)

Len 3396458


In [20]:
df

,COMPANY_KEY,COMPANY_CDE,SUB_BRANCH_KEY,SUB_BRANCH_CDE,CARD_KEY,CARD_CDE,CARDHDR_NO,LATEST_CARDNO,CUSTOMER_KEY,CUSTOMER_CDE,...,AMT_BILL,AMT_TXN_LCL,AMT_FEE,FOREX_MARKUP_AMT,MULT_NUM,MULT_CNT,PROCESS_DT,REC_CREATE_DT,REC_UPDATE_DT,ROW_WID
0,1,VNI,92,VN0011404,18106653,2638616769,422151XXXXXX4041,-1,31733437,13122286,...,501000.0,0.0,0.0,0.0,NaN,NaN,2021-03-02,2021-03-03 07:32:01,2021-03-03 07:32:01,None
1,1,VNI,285,VN0013111,35873635,2099521227,517416XXXXXX4176,-1,31657614,13084370,...,4300000.0,0.0,0.0,0.0,NaN,NaN,2021-03-02,2021-03-03 07:32:01,2021-03-03 07:32:01,None
2,1,VNI,310,VN0011163,1443235,3218556417,436438XXXXXX8121,None,26194458,54972,...,2000000.0,2000000.0,0.0,0.0,NaN,NaN,2020-05-02,2020-05-03 14:59:54,2020-05-03 14:59:54,None
3,1,VNI,310,VN0011163,1443235,3218556417,436438XXXXXX8121,None,26194458,54972,...,80000.0,80000.0,0.0,0.0,NaN,NaN,2020-05-02,2020-05-03 14:59:54,2020-05-03 14:59:54,None
4,1,VNI,78,VN0011353,24704467,2792638773,970403XXXXXX9433,-1,34806206,10275709,...,3001000.0,0.0,0.0,0.0,NaN,NaN,2021-03-31,2021-04-01 10:49:36,2021-04-01 10:49:36,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3396453,1,VNI,105,VN0011469,551434,2477676783,970403XXXXXX5816,970403XXXXXX5816,3747917,3794127,...,66000.0,0.0,0.0,0.0,0.0,0.0,2022-03-31,2022-04-01 15:04:57,2022-04-01 15:04:57,None
3396454,1,VNI,497,VN0014332,49587917,3434693572,436438XXXXXX2889,None,37530354,3668439,...,-8850000.0,NaN,0.0,0.0,NaN,NaN,2022-11-30,2022-12-01 07:12:25,2022-12-01 07:12:25,None
3396455,1,VNI,497,VN0014332,49587917,3434693572,436438XXXXXX2889,None,37530354,3668439,...,286900.0,NaN,0.0,0.0,NaN,NaN,2022-11-30,2022-12-01 07:12:25,2022-12-01 07:12:25,None
3396456,1,VNI,334,VN0011281,49607293,3640591784,436438XXXXXX5516,None,37446956,15698608,...,195900.0,NaN,0.0,0.0,NaN,NaN,2022-11-30,2022-12-01 07:12:25,2022-12-01 07:12:25,None


In [21]:
data_stats = gen_info_stats(df)
data_stats

,Column,Null Pct,Dtype,Nunique,Nunique Pct,Sample Unique Value
0,COMPANY_KEY,0.000000e+00,int64,3,8.832731e-07,"[1, 2, 3]"
1,COMPANY_CDE,0.000000e+00,object,3,8.832731e-07,"[VNI, LAO, CAM]"
2,SUB_BRANCH_KEY,0.000000e+00,int64,588,1.731215e-04,"[92, 285, 310]"
3,SUB_BRANCH_CDE,0.000000e+00,object,592,1.742992e-04,"[VN0011404, VN0013111, VN0011163]"
4,CARD_KEY,0.000000e+00,int64,308639,9.087084e-02,"[18106653, 35873635, 1443235]"
5,CARD_CDE,0.000000e+00,object,213483,6.285460e-02,"[2638616769, 2099521227, 3218556417]"
6,CARDHDR_NO,0.000000e+00,object,50870,1.497737e-02,"[422151XXXXXX4041, 517416XXXXXX4176, 436438XXX..."
7,LATEST_CARDNO,2.360400e-01,object,136784,4.027254e-02,"[-1, None, 422151XXXXXX7886 ]"
8,CUSTOMER_KEY,0.000000e+00,int64,208252,6.131446e-02,"[31733437, 31657614, 26194458]"
9,CUSTOMER_CDE,0.000000e+00,object,158490,4.666332e-02,"[13122286, 13084370, 54972]"


In [22]:
df.describe()

,COMPANY_KEY,SUB_BRANCH_KEY,CARD_KEY,CUSTOMER_KEY,DAO_KEY,PRODUCT_KEY,MCC_CDE,AMT_TXN,AMT_BILL,AMT_TXN_LCL,AMT_FEE,FOREX_MARKUP_AMT,MULT_NUM,MULT_CNT
count,3.396458e+06,3.396458e+06,3.396458e+06,3.396458e+06,3.396458e+06,3.396458e+06,3.396458e+06,3.396458e+06,3.396458e+06,3.374621e+06,3.396458e+06,3.396458e+06,1.143548e+06,1.143548e+06
mean,1.000510e+00,2.624593e+02,2.913386e+07,3.235633e+07,4.422786e+04,1.499226e+03,4.812266e+03,2.774002e+06,2.469249e+06,5.698290e+05,6.374608e+02,9.918415e+01,1.281450e-02,1.514934e-02
std,2.761509e-02,1.701962e+02,1.286714e+07,6.752892e+06,8.935630e+03,8.838116e+02,3.492185e+03,6.959163e+06,7.068582e+06,5.037430e+06,1.590395e+04,6.925646e+03,1.125746e-01,4.891193e-01
min,1.000000e+00,-1.000000e+00,1.005000e+03,1.258630e+05,-1.000000e+00,-1.000000e+00,0.000000e+00,-1.283908e+08,-7.292933e+08,-1.283908e+08,-9.689410e+05,-3.214127e+05,0.000000e+00,0.000000e+00
25%,1.000000e+00,1.180000e+02,2.010679e+07,2.878967e+07,3.891100e+04,1.830000e+03,0.000000e+00,1.090000e+05,1.000000e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.000000e+00,2.560000e+02,2.748567e+07,3.327048e+07,4.420000e+04,1.851000e+03,6.011000e+03,7.417000e+05,6.380000e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.000000e+00,3.760000e+02,3.925458e+07,3.617375e+07,5.054000e+04,1.913000e+03,6.011000e+03,3.000000e+06,3.000000e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,3.000000e+00,7.150000e+02,6.759788e+07,4.698172e+07,6.971400e+04,5.041000e+03,9.999000e+03,2.210000e+09,2.210000e+09,9.868969e+08,9.727016e+06,5.436050e+06,4.000000e+00,9.900000e+01


## EDA

Selected some col data to EDA, possible these col are good to go, (can be customized):
- Category data:
    - Unique Pct <= x (too many leading data can be identified easily)
    - Nunique > 1 (=1 doesn't mean anything)
- Numerical data: good to select all

In [23]:
valid_info = None 
try: 
    valid_info = data_stats[((data_stats['Nunique Pct'] <= 0.3) & (data_stats['Nunique']>1) & (data_stats['Dtype']==object)) | (data_stats['Dtype']!=object)]
    display(valid_info)
except Exception as e:
    pass

,Column,Null Pct,Dtype,Nunique,Nunique Pct,Sample Unique Value
0,COMPANY_KEY,0.000000e+00,int64,3,8.832731e-07,"[1, 2, 3]"
1,COMPANY_CDE,0.000000e+00,object,3,8.832731e-07,"[VNI, LAO, CAM]"
2,SUB_BRANCH_KEY,0.000000e+00,int64,588,1.731215e-04,"[92, 285, 310]"
3,SUB_BRANCH_CDE,0.000000e+00,object,592,1.742992e-04,"[VN0011404, VN0013111, VN0011163]"
4,CARD_KEY,0.000000e+00,int64,308639,9.087084e-02,"[18106653, 35873635, 1443235]"
5,CARD_CDE,0.000000e+00,object,213483,6.285460e-02,"[2638616769, 2099521227, 3218556417]"
6,CARDHDR_NO,0.000000e+00,object,50870,1.497737e-02,"[422151XXXXXX4041, 517416XXXXXX4176, 436438XXX..."
7,LATEST_CARDNO,2.360400e-01,object,136784,4.027254e-02,"[-1, None, 422151XXXXXX7886 ]"
8,CUSTOMER_KEY,0.000000e+00,int64,208252,6.131446e-02,"[31733437, 31657614, 26194458]"
9,CUSTOMER_CDE,0.000000e+00,object,158490,4.666332e-02,"[13122286, 13084370, 54972]"


In [24]:
df.columns, len(df.columns)

(Index(['COMPANY_KEY', 'COMPANY_CDE', 'SUB_BRANCH_KEY', 'SUB_BRANCH_CDE',
        'CARD_KEY', 'CARD_CDE', 'CARDHDR_NO', 'LATEST_CARDNO', 'CUSTOMER_KEY',
        'CUSTOMER_CDE', 'SUB_SECTOR_CDE', 'DAO_KEY', 'DAO_CDE',
        'SUB_INDUSTRY_CDE', 'DECISION_CDE', 'MERCHANT_CDE', 'TXN_CURR_CDE',
        'BILL_CURR_CDE', 'PRODUCT_KEY', 'PRODUCT_CDE', 'MCC_CDE',
        'POS_COND_CDE', 'POSMODE_CDE', 'ACQ_CNTRY_CDE', 'TERMINAL_ID',
        'RESPONSE_CDE', 'TXN_OL_CDE', 'TXN_OM_CDE', 'TRXN_CAT', 'REFERRAL_CDE',
        'RETRVL_REFNO', 'ACCOUNT_NO', 'FIN_ACCTNO', 'APPROVAL_CDE',
        'ACQUIRER_REFNO', 'POST_DT', 'REQUEST_DT', 'REQUEST_TIME',
        'PARTICULARS', 'OFFICER_ID', 'SOURCE_CHANNEL', 'POS_INFO',
        'TRAN_STATUS', 'MATCH_FLAG', 'SRC_IND', 'TRXN_ID', 'TID_TRAN_IND',
        'ACCOUNT_CREDIT', 'AMT_TXN', 'AMT_BILL', 'AMT_TXN_LCL', 'AMT_FEE',
        'FOREX_MARKUP_AMT', 'MULT_NUM', 'MULT_CNT', 'PROCESS_DT',
        'REC_CREATE_DT', 'REC_UPDATE_DT', 'ROW_WID'],
       dtype='obje

In [25]:
valid_info.set_index('Column').index, len(valid_info['Column'].index)

(Index(['COMPANY_KEY', 'COMPANY_CDE', 'SUB_BRANCH_KEY', 'SUB_BRANCH_CDE',
        'CARD_KEY', 'CARD_CDE', 'CARDHDR_NO', 'LATEST_CARDNO', 'CUSTOMER_KEY',
        'CUSTOMER_CDE', 'SUB_SECTOR_CDE', 'DAO_KEY', 'DAO_CDE',
        'SUB_INDUSTRY_CDE', 'DECISION_CDE', 'MERCHANT_CDE', 'TXN_CURR_CDE',
        'BILL_CURR_CDE', 'PRODUCT_KEY', 'PRODUCT_CDE', 'MCC_CDE',
        'POS_COND_CDE', 'POSMODE_CDE', 'ACQ_CNTRY_CDE', 'TERMINAL_ID',
        'RESPONSE_CDE', 'TXN_OL_CDE', 'TXN_OM_CDE', 'TRXN_CAT', 'REFERRAL_CDE',
        'ACCOUNT_NO', 'FIN_ACCTNO', 'APPROVAL_CDE', 'ACQUIRER_REFNO', 'POST_DT',
        'REQUEST_DT', 'REQUEST_TIME', 'PARTICULARS', 'OFFICER_ID',
        'SOURCE_CHANNEL', 'POS_INFO', 'TRAN_STATUS', 'MATCH_FLAG', 'SRC_IND',
        'TRXN_ID', 'TID_TRAN_IND', 'ACCOUNT_CREDIT', 'AMT_TXN', 'AMT_BILL',
        'AMT_TXN_LCL', 'AMT_FEE', 'FOREX_MARKUP_AMT', 'MULT_NUM', 'MULT_CNT',
        'PROCESS_DT', 'REC_CREATE_DT', 'REC_UPDATE_DT', 'ROW_WID'],
       dtype='object', name='Column'),
 58

## COMPARE SAMPLES - SUB1/2/3

In [26]:
subsamples = sorted(os.listdir(NAMESPACE.IMG_FP))
subsamples[:3]

['SAMPLE1_SUB1', 'SAMPLE1_SUB3']

In [27]:
all_img_names = []
for sub in subsamples:
    sub_fp = os.path.join(NAMESPACE.IMG_FP, sub)
    img_names = os.listdir(sub_fp)
    img_names = [i for i in img_names if i.endswith('jpg')]
    all_img_names.extend(img_names)
all_img_names = list(set(all_img_names))
all_img_names[:3]

[]

In [28]:
for img_nm in img_names[:]:
    display(img_nm)
    fig, axes = plt.subplots(len(subsamples),1,figsize=(15,15))
    for i, sub in enumerate(subsamples):
        img_fp = os.path.join(NAMESPACE.IMG_FP, sub, img_nm)
        img = mpimg.imread(img_fp)
        axes[i].imshow(img)
    plt.show()